### Root Imaging
Test notebook for root imaging

In [1]:
# import required modules
import sys
sys.path.append('..')
from utils.MachineUtils import *
import time
import random
import numpy as np
import cv2 as cv2
import utils.PlatePositionUtils as pp 
from utils.CameraUtils import *
import json

In [2]:
# Setup your machine connection
# List available ports in thie cell

ports = serial.tools.list_ports.comports()
print([port.name for port in ports]) 

['ttyACM0', 'ttyAMA0']


In [3]:
# Choose the correct port from above and establish connection with machine
# (if emergency stopped, change last number between 0 and 1)
port = '/dev/ttyACM0'
m = MachineCommunication(port)

In [4]:
# important machine commands!
# m.moveTo() -- absolute move
# m.move() == relative
# m.toolChange(), -1 for no tool 

In [5]:
# Place duckweed photobooth into plate position 6
# place 24 wells into plate position 1-3
# place petri dish with water into position 4

In [7]:
# important camera commands!
# f = getFrame() take a pic
# showFrame(f) show
# showFrame(f, grid=True)pic.jpg
# saveFrame(f, '~/Downloads/')

In [ ]:
# well plate @1
# photobooth @2
# water @ 5

In [5]:
# pickup LH syringe
m.toolChange(2)

In [6]:
# moves water into photo changer
m.moveTo(x=225, y=45)
m.moveTo(z=-42)
m.move(de=15, s=1000)
m.moveTo(z=0)
m.moveTo(x=75.5, y=47)
m.move(de=-10, s=1000)
m.moveTo(x=225, y=45)
m.move(de=-5, s=1000)
m.moveTo(z=100)

In [7]:
# find the water height manually 
m.toolChange(3)

well = pp.fetch_well_position(1, 'A1')

m.moveTo(x=well['x'], y=well['y'])


In [88]:
# user changes well plate frond height for 10cc syringe

# Change me!
water_height = -10.2

In [89]:
# camera is currently calibrated at z=80 to translate to machine coordinates
m.moveTo(z=80)

In [90]:
# 
m.toolChange(1)
m.moveTo(x=well['x'], y=well['y'])

In [11]:
# loads in the relevant calibration files

# with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_60_tape.json") as f:
with open("/home/pi/autofocus-test/JubileeAutofocus/camera_cal_z_80.json") as f:
    cal = json.load(f)
matrix = np.array(cal['transform'])
size = cal['resolution']
print(matrix)
print(size)

m.transform = matrix
m.img_size = size

[[ -0.7296065   -0.35246752]
 [  0.38097134  -0.71682782]
 [ -0.32375357   0.6160794 ]
 [  1.60644767  83.44887212]
 [-82.50757965   1.39357274]
 [ 75.86369137 238.95022829]]
[1200, 1200]


In [12]:
#focus camera
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    center = (int(w/2), int(h/2))
    print(center)

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

(320, 240)


In [91]:
f = getFrame()
pts = selectPoint(f, num_pts=1)

[(635.2142857142858, 549.4999999999998)]


Exception in Tkinter callback
Traceback (most recent call last):
  File "/usr/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/usr/lib/python3.7/tkinter/__init__.py", line 749, in callit
    func(*args)
  File "/home/pi/duckbot/.venv/lib/python3.7/site-packages/matplotlib/backends/_backend_tk.py", line 476, in delayed_destroy
    self.window.destroy()
  File "/usr/lib/python3.7/tkinter/__init__.py", line 2061, in destroy
    for c in list(self.children.values()): c.destroy()
  File "/usr/lib/python3.7/tkinter/__init__.py", line 2308, in destroy
    Misc.destroy(self)
  File "/usr/lib/python3.7/tkinter/__init__.py", line 592, in destroy
    self.tk.deletecommand(name)
_tkinter.TclError: can't delete Tcl command


In [92]:
# convert px coord to real coord
def px_to_real(x,y, absolute = False):
        x = (x / m.img_size[0]) - 0.5
        y = (y / m.img_size[1]) - 0.5
        a = 1 if absolute else 0

        return (m.transform.T @ np.array([x**2, y**2, x * y, x, y, a]))

fronds = []
for pt in pts:
    frond_off = px_to_real(pt[0], pt[1])
    frond = [well['x'] - frond_off[0], well['y'] - frond_off[1]]
    fronds.append(frond)

In [93]:
# to increase syringe speed, send:
# M201 E2500
# M203 E2500
m.toolChange(3)
m.moveTo(x=well['x'], y=well['y'])

In [94]:
idx = 0
fx = fronds[idx][0]
fy = fronds[idx][1]
fx, fy

(25.480218208296126, 172.6121530214497)

In [95]:
syringe_off = [-0.76, 4.2] # pink
m.moveTo(m.moveTo(x=fx + syringe_off[0], y=fy + syringe_off[1]))

In [18]:
def aspirate_2():
#     m.move(de=5, s=1000) ; # 'prime' syringe
    m.moveTo(z=water_height + 2)
    m.dwell(1000)
    m.move(dz=-3.5, de=5) # press slightly
#     m.dwell(250)
    m.move(de=40, s=1800) # aspirate! #1200 for lemna minor
    m.move(dz=6, s=500, de=5) # aspirate!
#     m.move(dz=5, de=50, s=1000) # aspirate!
    
def dispense():
    m.dwell(1000)
    m.move(de=-50, s=1400) # dispense
    m.dwell(500)

In [96]:

aspirate_2()
m.moveTo(z=45)
m.moveTo(x=75.5, y=47)


In [63]:
m.moveTo(z=6)
m.move(de=-12, s=200)
m.moveTo(z=20)
m.dwell(1000)
m.moveTo(z=10)
m.moveTo(z=20)
m.dwell(1000)
m.moveTo(z=10)
m.moveTo(z=20)
m.dwell(1000)
m.moveTo(z=10)
m.moveTo(z=20)
m.dwell(1000)
m.moveTo(z=10)
m.moveTo(z=20)
m.dwell(1000)
m.moveTo(z=10)
m.moveTo(z=45)

In [97]:
m.moveTo(x=225, y=45)

In [76]:

m.move(de=-35, s=1000)

In [98]:
# no dw
dispense()

In [5]:
m.toolChange(4)

In [53]:
m.moveTo(x=79, y=87)
m.moveTo(z=-55.2)

In [64]:
cap = cv2.VideoCapture(0) #Note that the index corresponding to your camera may not be zero but this is the most common default

# draw a circle in the center of the frame
center = None
while center is None:
    # the first frame grab is sometimes empty
    ret, frame = cap.read()
    h, w = frame.shape[0:2]
    center = (int(w/2), int(h/2))
    print(center)

while True:
    ret, frame = cap.read()
    target = cv2.circle(frame, center, 5, (0,255,0), -1)
    cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c ==27: #27 is the built in code for ESC so press escape to close the window. 
        break 
        
cap.release()
cv2.destroyAllWindows()

(320, 240)


In [65]:
f = getFrame()
showFrame(f)

In [66]:
saveFrame(f, '/home/pi/Downloads/root4pink-backlight.jpeg')

In [23]:
m.moveTo(x=well['x'], y=well['y'])

In [24]:
dispense()